In [25]:
import sys
sys.path.append('/usr/app/frustum-pointnets/train/')
import provider
import numpy as np

pc_camera = np.asarray([
    [1,0,3],
    [2,0,3],
    [3,0,3],
    [2,0,2],
    [3,0,2],
    [3,0,1],
])
h,w,l,tx,ty,tz,ry = [2,1,2,
                     2,-1,2,
                     0]
gt_box3d = np.array([h,w,l,tx,ty,tz,ry])

In [30]:
def compute_obj_coordinate(pc_, box3d, norm=False):
    '''
    transform the pc from camera frame to the object frame specified by box3d
    inputs:
        pc: numpy.array [#of points, >=3]
            coordinates of the points in the camera frame
        box3d: numpy.array [7,] ([h,w,l,tx,ty,tz,ry])
            3d bounding box
    outputs:
        numpy.array [#of points, 3]
    '''
    pc = np.copy(pc_[:,:3])
    h,w,l,tx,ty,tz,ry = box3d
    bottom_center = np.array([tx, ty, tz])
    center = np.array([tx, ty+h/2.0, tz])
    box_size = np.array([h,w,l])
    pc_obj = provider.rotate_pc_along_y(pc - center, -ry)
    if norm == True:
        pc_obj = pc_obj / (box_size / 2.0) # normalize to [-1 , 1]
        pc_obj = (pc_obj + 1 ) / 2.0 # normalize to [0 , 1]
    return pc_obj
        

def recover_bottom_center(pc_camera, pc_obj_, box_size, ry):
    '''
    recover the center point coordinate with the equation:
        p_o^c = p_i^c - R_o^c * p_i^o
    inputs:
        pc_camera: numpy.array [#of points, >=3]
            coordinates of the points in the camera frame
        pc_obj: numpy.array [#of points, >=3]
            coordinates of the points in the object frame
        box_size: numpy.array [3,] ([h,wl])
        ry: float (rad)
    outputs:
        numpy.array [#of points, 3]
    '''
    pc_obj = np.copy(pc_obj_)
    pc_obj = pc_obj * 2.0 - 1
    pc_obj = pc_obj * (box_size / 2.0)
    h,_,_ = box_size
    tmp = provider.rotate_pc_along_y(np.copy(pc_obj), ry)
    return pc_camera - tmp - np.array([0,h/2.0,0])


In [31]:
pc_object = compute_obj_coordinate(pc_camera, gt_box3d, norm=True)

In [32]:
gt_center = recover_bottom_center(pc_camera, pc_object, np.array([h,w,l]), ry)

In [33]:
gt_center

array([[ 2., -1.,  2.],
       [ 2., -1.,  2.],
       [ 2., -1.,  2.],
       [ 2., -1.,  2.],
       [ 2., -1.,  2.],
       [ 2., -1.,  2.]])

In [15]:
pc_object

array([[ 0. ,  0.5,  1. ],
       [ 0.5,  0.5,  1. ],
       [ 1. ,  0.5,  1. ],
       [ 0.5,  0.5,  0.5],
       [ 1. ,  0.5,  0.5],
       [ 1. ,  0.5,  0. ]])